In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/myModel

/content/drive/My Drive/myModel


In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
import sklearn
import resnet
import os
from keras.callbacks import EarlyStopping
import keras

## **데이터 불러오기**

#### train set과 test set 을 따로 만들었을 경우에는 이 코드를 사용

In [30]:
train_data = np.loadtxt('./train.csv', delimiter=',')
test_data = np.loadtxt('./test.csv', delimiter=',')

x_train = train_data[:,1:]
y_train = train_data[:,0]
x_test = test_data[:,1:]
y_test = test_data[:,0]

# znorm
std_ = x_train.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_
std_ = x_test.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_

print("x_train:", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train", y_train.shape)
print("y_test: ", y_test.shape)

x_train: (270, 1000)
x_test:  (120, 1000)
y_train (270,)
y_test:  (120,)


#### 전체 dataset에서 케라스가 알아서 train과 test로 나누고 싶을 때 이 코드 사용

In [ ]:
def read_dataset(root_dir, dataset_name):
    datasets_dict = {}

    df = np.loadtxt('./ALL_data.csv', delimiter=',')

    x_train, x_test, y_train, y_test = train_test_split(df[:,1:], df[:,0], test_size=0.3)
    print("x_train:", x_train.shape)
    print("x_test: ", x_test.shape)
    print("y_train", y_train.shape)
    print("y_test: ", y_test.shape)

    # znorm
    std_ = x_train.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_
    std_ = x_test.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_
    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                    y_test.copy())
    return datasets_dict

datasets_dict= read_dataset('', 'handcream')

x_train: (1770, 2000)
x_test:  (759, 2000)
y_train (1770,)
y_test:  (759,)


## **폴더 만들기**
#### 모델 실행 후 만들어질 파일들이 저장될거임

In [31]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            return None
        return directory_path

## **원핫인코딩**

In [32]:
dataset_name = 'handcream'

# x_train = datasets_dict[dataset_name][0]
# y_train = datasets_dict[dataset_name][1]
# x_test = datasets_dict[dataset_name][2]
# y_test = datasets_dict[dataset_name][3]
print("x_train:", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train", y_train.shape)
print("y_test: ", y_test.shape)

nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
print("class:", nb_classes)
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

y_true = np.argmax(y_test, axis=1)

if len(x_train.shape) == 2:
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]

output_directory = './resnet_' + dataset_name + '/'
create_directory(output_directory)

x_train: (270, 1000)
x_test:  (120, 1000)
y_train (270,)
y_test:  (120,)
class: 3


## **ResNet 모델**

In [36]:
n_feature_maps = 64

input_layer = keras.layers.Input(input_shape)

# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=32, padding='same')(input_layer) #8
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=16, padding='same')(conv_x) #5
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(conv_y) #3
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=32, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=16, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
print("output_block_2:", output_block_2)
output_block_2 = keras.layers.Activation('relu')(output_block_2)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=32, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=16, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)
# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

file_path = output_directory + 'best_model.hdf5'

early_stopping_callback = EarlyStopping(monitor = 'loss', mode='min', patience=20, verbose=1)

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                    save_best_only=True)

callbacks = [reduce_lr, model_checkpoint, early_stopping_callback]


output_block_2: KerasTensor(type_spec=TensorSpec(shape=(None, 1000, 128), dtype=tf.float32, name=None), name='add_7/add:0', description="created by layer 'add_7'")


In [37]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 1000, 64)     2112        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_24 (BatchNo (None, 1000, 64)     256         conv1d_22[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 1000, 64)     0           batch_normalization_24[0][0]     
____________________________________________________________________________________________

## **학습**

In [38]:
# validation_split: train dataset에서 validation dataset으로 분리할 크기 
# 0.2 => 20%
history = model.fit(x_train, y_train, batch_size=16, epochs=100, validation_split=0.3, callbacks=callbacks)
self.model.save(output_directory + 'last_model.hdf5')

Epoch 1/100
12/12 [==============================] - 7s 161ms/step - loss: 0.6132 - accuracy: 0.5975 - val_loss: 14.3209 - val_accuracy: 0.3704


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
12/12 [==============================] - 1s 65ms/step - loss: 0.2535 - accuracy: 0.8889 - val_loss: 6.5689 - val_accuracy: 0.3704
Epoch 3/100
12/12 [==============================] - 1s 71ms/step - loss: 0.1617 - accuracy: 0.9466 - val_loss: 10.8373 - val_accuracy: 0.3704
Epoch 4/100
12/12 [==============================] - 1s 69ms/step - loss: 0.2099 - accuracy: 0.9398 - val_loss: 3.5891 - val_accuracy: 0.7531
Epoch 5/100
12/12 [==============================] - 1s 73ms/step - loss: 0.0785 - accuracy: 0.9763 - val_loss: 4.5135 - val_accuracy: 0.7531
Epoch 6/100
12/12 [==============================] - 1s 67ms/step - loss: 0.0876 - accuracy: 0.9748 - val_loss: 2.5404 - val_accuracy: 0.7407
Epoch 7/100
12/12 [==============================] - 1s 66ms/step - loss: 0.3162 - accuracy: 0.8924 - val_loss: 8.7316 - val_accuracy: 0.3704
Epoch 8/100
12/12 [==============================] - 1s 66ms/step - loss: 0.1333 - accuracy: 0.9647 - val_loss: 8.8854 - val_accuracy: 0.3704
Epoch

NameError: ignored

## **모델 평가**

In [39]:
import matplotlib.pyplot as plt
import keras
import sklearn

model_path = output_directory + 'best_model.hdf5'
model = keras.models.load_model(model_path)

print("정확도 : %.2f " %(model.evaluate(x_test, y_test)[1]))

prediction = model.predict(x_test)

for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)

y_pred = model.predict(x_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)

4/4 [==============================] - 1s 31ms/step - loss: 0.2297 - accuracy: 0.9083
정확도 : 0.91 
[1.4792218e-04 9.9896157e-01 8.9056639e-04]
1
[1.1922223e-05 9.9996793e-01 2.0180274e-05]
1
[9.99881387e-01 1.14341274e-04 4.33535251e-06]
0
[9.9154752e-01 8.4291175e-03 2.3431661e-05]
0
[9.996792e-01 3.129339e-04 7.842198e-06]
0
[9.993772e-01 5.448522e-04 7.808196e-05]
0
[4.7806323e-05 1.5801768e-04 9.9979419e-01]
2
[4.9650724e-05 9.9992943e-01 2.0866555e-05]
1
[9.1781667e-06 9.9996686e-01 2.3978198e-05]
1
[9.9833196e-01 1.6568702e-03 1.1203174e-05]
0
[0.00213858 0.97460073 0.02326066]
1
[2.4805146e-05 1.0269873e-04 9.9987245e-01]
2
[0.00546529 0.50929075 0.48524395]
1
[1.7183952e-04 1.6827103e-03 9.9814546e-01]
2
[0.00309009 0.8503266  0.14658336]
1
[9.8530654e-06 9.9998295e-01 7.1590621e-06]
1
[1.5891877e-04 9.9964118e-01 1.9979051e-04]
1
[3.3689651e-04 6.5553922e-04 9.9900764e-01]
2
[0.00102517 0.00430938 0.9946654 ]
2
[9.9951375e-01 4.2940155e-04 5.6783963e-05]
0
[9.7957039e-01 4.8522

## **시각화**

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.savefig('loss.png')
plt.clf()
plt.plot(history.history['val_loss'])
plt.savefig('vloss.png')
plt.clf()
plt.plot(history.history['accuracy'])
plt.savefig('acc.png')
plt.clf()
plt.plot(history.history['val_accuracy'])
plt.savefig('vacc.png')
plt.clf()
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'accuracy', 'val_accuracy'], loc='upper left')
# plt.show()